# LangGraph Document Components và API Reference

## Mục tiêu học tập
- Hiểu về cấu trúc tài liệu của LangGraph và cách sử dụng API Reference
- Nắm vững các thành phần chính: Nodes, Edges, State, Graph
- Thực hành với các API quan trọng của LangGraph
- Xây dựng các ví dụ thực tế sử dụng ChatAnthropic

## Giới thiệu

LangGraph là một framework mạnh mẽ để xây dựng các ứng dụng AI phức tạp dựa trên đồ thị (graph). Các thành phần chính của LangGraph bao gồm:

### 🎯 Thành phần cơ bản
1. **Graph**: Container chính chứa toàn bộ workflow
2. **StateGraph**: Graph có khả năng quản lý state
3. **Node**: Các điểm xử lý logic (functions/agents)
4. **Edge**: Kết nối giữa các nodes
5. **State**: Dữ liệu được chia sẻ giữa các nodes

### 🔗 API Methods quan trọng
- `add_node()`: Thêm node vào graph
- `add_edge()`: Tạo kết nối giữa các nodes
- `set_entry_point()`: Xác định điểm bắt đầu
- `compile()`: Biên dịch graph thành runnable
- `invoke()`: Thực thi đồng bộ
- `stream()`: Thực thi bất đồng bộ với streaming

## Cài đặt và Import

In [ ]:
# Cài đặt các package cần thiết
!pip install langgraph langchain-anthropic python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Dict, Any
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool
import json

# Load environment variables
load_dotenv()

# Kiểm tra API key
if not os.getenv("ANTHROPIC_API_KEY"):
    print("⚠️ ANTHROPIC_API_KEY chưa được thiết lập!")
else:
    print("✅ ANTHROPIC_API_KEY đã được thiết lập")

## API Reference và Cấu trúc Tài liệu

### 📚 Tài liệu chính thức LangGraph
- **API Reference**: https://langchain-ai.github.io/langgraph/reference/
- **Hướng dẫn**: https://langchain-ai.github.io/langgraph/
- **Examples**: https://github.com/langchain-ai/langgraph/tree/main/examples

### 🔍 Cách sử dụng API Reference
1. Truy cập trang API Reference chính thức
2. Tìm kiếm class/method cần sử dụng
3. Đọc docstring và parameters
4. Xem examples và return types
5. Kiểm tra version compatibility

## Ví dụ 1: Graph Đơn giản với StateGraph

In [ ]:
# Định nghĩa State schema
class SimpleState(TypedDict):
    messages: Annotated[list, add_messages]
    step_count: int
    current_action: str

# Khởi tạo LLM
llm = ChatAnthropic(
    model="claude-3-haiku-20240307",
    temperature=0,
    max_tokens=1000
)

def greeting_node(state: SimpleState) -> Dict[str, Any]:
    """Node chào hỏi đầu tiên"""
    print(f"🔸 Executing greeting_node - Step: {state.get('step_count', 0)}")
    
    response = llm.invoke([
        HumanMessage(content="Chào bạn! Hãy giới thiệu bản thân một cách ngắn gọn.")
    ])
    
    return {
        "messages": [response],
        "step_count": state.get('step_count', 0) + 1,
        "current_action": "greeting_completed"
    }

def analysis_node(state: SimpleState) -> Dict[str, Any]:
    """Node phân tích thông tin"""
    print(f"🔸 Executing analysis_node - Step: {state.get('step_count', 0)}")
    
    # Lấy tin nhắn cuối cùng từ state
    last_message = state["messages"][-1].content if state["messages"] else ""
    
    response = llm.invoke([
        HumanMessage(content=f"Phân tích ngắn gọn về nội dung này: {last_message}")
    ])
    
    return {
        "messages": [response],
        "step_count": state.get('step_count', 0) + 1,
        "current_action": "analysis_completed"
    }

def summary_node(state: SimpleState) -> Dict[str, Any]:
    """Node tóm tắt cuối cùng"""
    print(f"🔸 Executing summary_node - Step: {state.get('step_count', 0)}")
    
    total_messages = len(state["messages"])
    
    response = llm.invoke([
        HumanMessage(content=f"Tóm tắt cuộc trò chuyện với {total_messages} tin nhắn đã xử lý.")
    ])
    
    return {
        "messages": [response],
        "step_count": state.get('step_count', 0) + 1,
        "current_action": "summary_completed"
    }

# Tạo StateGraph
workflow = StateGraph(SimpleState)

# Thêm các nodes
workflow.add_node("greeting", greeting_node)
workflow.add_node("analysis", analysis_node)
workflow.add_node("summary", summary_node)

# Thiết lập entry point
workflow.set_entry_point("greeting")

# Thêm các edges
workflow.add_edge("greeting", "analysis")
workflow.add_edge("analysis", "summary")
workflow.add_edge("summary", END)

# Compile graph
simple_app = workflow.compile()

print("✅ Graph đơn giản đã được tạo thành công!")

In [ ]:
# Thực thi graph đơn giản
print("🚀 Bắt đầu thực thi Graph đơn giản...\n")

initial_state = {
    "messages": [],
    "step_count": 0,
    "current_action": "starting"
}

result = simple_app.invoke(initial_state)

print("\n📊 Kết quả cuối cùng:")
print(f"- Tổng số bước: {result['step_count']}")
print(f"- Hành động cuối: {result['current_action']}")
print(f"- Số tin nhắn: {len(result['messages'])}")

print("\n💬 Tin nhắn cuối cùng:")
if result['messages']:
    print(result['messages'][-1].content)

## Ví dụ 2: Agent với Tool Integration

In [ ]:
# Định nghĩa tools
@tool
def calculate_math(expression: str) -> str:
    """Tính toán biểu thức toán học đơn giản.
    
    Args:
        expression: Biểu thức toán học (ví dụ: '2+3*4')
    
    Returns:
        Kết quả tính toán
    """
    try:
        # Chỉ cho phép các ký tự an toàn
        allowed_chars = set('0123456789+-*/(). ')
        if not all(char in allowed_chars for char in expression):
            return "Lỗi: Biểu thức chứa ký tự không hợp lệ"
        
        result = eval(expression)
        return f"Kết quả của {expression} = {result}"
    except Exception as e:
        return f"Lỗi tính toán: {str(e)}"

@tool
def get_weather_info(city: str) -> str:
    """Lấy thông tin thời tiết (giả lập).
    
    Args:
        city: Tên thành phố
    
    Returns:
        Thông tin thời tiết
    """
    weather_data = {
        "hà nội": "Hà Nội: 25°C, nắng ít mây, độ ẩm 60%",
        "hồ chí minh": "TP.HCM: 30°C, mưa rào, độ ẩm 80%",
        "đà nẵng": "Đà Nẵng: 28°C, nắng đẹp, độ ẩm 55%"
    }
    
    city_lower = city.lower()
    return weather_data.get(city_lower, f"Không có dữ liệu thời tiết cho {city}")

# Tạo LLM với tools
tools = [calculate_math, get_weather_info]
llm_with_tools = llm.bind_tools(tools)

print("🛠️ Tools đã được định nghĩa:")
for tool in tools:
    print(f"- {tool.name}: {tool.description}")

In [ ]:
# State cho Agent
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    tool_calls_made: int
    current_task: str

def agent_node(state: AgentState) -> Dict[str, Any]:
    """Node agent chính"""
    print(f"🤖 Agent đang xử lý...")
    
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    
    return {
        "messages": [response],
        "current_task": "agent_processing"
    }

def tool_node(state: AgentState) -> Dict[str, Any]:
    """Node thực thi tools"""
    print(f"🔧 Thực thi tools...")
    
    messages = state["messages"]
    last_message = messages[-1]
    
    tool_calls = getattr(last_message, 'tool_calls', [])
    
    if not tool_calls:
        return {"current_task": "no_tools"}
    
    results = []
    tools_dict = {tool.name: tool for tool in tools}
    
    for tool_call in tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        
        if tool_name in tools_dict:
            try:
                result = tools_dict[tool_name].invoke(tool_args)
                results.append({
                    "tool_call_id": tool_call['id'],
                    "name": tool_name,
                    "content": result
                })
                print(f"✅ Tool {tool_name} thực thi thành công")
            except Exception as e:
                results.append({
                    "tool_call_id": tool_call['id'],
                    "name": tool_name,
                    "content": f"Lỗi: {str(e)}"
                })
                print(f"❌ Tool {tool_name} gặp lỗi: {str(e)}")
    
    return {
        "messages": results,
        "tool_calls_made": state.get('tool_calls_made', 0) + len(results),
        "current_task": "tools_executed"
    }

def should_continue(state: AgentState) -> str:
    """Quyết định có tiếp tục hay không"""
    messages = state["messages"]
    last_message = messages[-1]
    
    # Nếu có tool calls, thực thi tools
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    else:
        return "end"

# Tạo Agent Graph
agent_workflow = StateGraph(AgentState)

# Thêm nodes
agent_workflow.add_node("agent", agent_node)
agent_workflow.add_node("tools", tool_node)

# Entry point
agent_workflow.set_entry_point("agent")

# Conditional edges
agent_workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "end": END
    }
)

# Tools node quay lại agent
agent_workflow.add_edge("tools", "agent")

# Compile
agent_app = agent_workflow.compile()

print("✅ Agent Graph với Tools đã được tạo thành công!")

In [ ]:
# Test Agent với Tools
print("🚀 Test Agent với Tools...\n")

test_queries = [
    "Tính toán 15 * 8 + 32 cho tôi",
    "Thời tiết ở Hà Nội hôm nay như thế nào?",
    "Vừa tính (100 - 25) / 5 vừa cho biết thời tiết Đà Nẵng"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*50}")
    print(f"📝 Test {i}: {query}")
    print(f"{'='*50}")
    
    initial_state = {
        "messages": [HumanMessage(content=query)],
        "tool_calls_made": 0,
        "current_task": "starting"
    }
    
    try:
        result = agent_app.invoke(initial_state)
        
        print(f"\n📊 Kết quả:")
        print(f"- Tool calls made: {result.get('tool_calls_made', 0)}")
        print(f"- Current task: {result.get('current_task', 'unknown')}")
        
        # Lấy phản hồi cuối cùng từ AI
        ai_messages = [msg for msg in result['messages'] if isinstance(msg, AIMessage)]
        if ai_messages:
            print(f"\n🤖 AI Response:")
            print(ai_messages[-1].content)
            
    except Exception as e:
        print(f"❌ Lỗi: {str(e)}")

## Ví dụ 3: Truy cập và Xử lý State

In [ ]:
# State phức tạp với nhiều thông tin
class ComplexState(TypedDict):
    messages: Annotated[list, add_messages]
    user_profile: Dict[str, Any]
    conversation_history: list
    current_topic: str
    processing_steps: list
    metadata: Dict[str, Any]

def profile_analyzer_node(state: ComplexState) -> Dict[str, Any]:
    """Phân tích thông tin user từ tin nhắn"""
    print("👤 Analyzing user profile...")
    
    # Lấy tin nhắn mới nhất từ user
    human_messages = [msg for msg in state["messages"] if isinstance(msg, HumanMessage)]
    if not human_messages:
        return {"processing_steps": ["No human messages found"]}
    
    latest_message = human_messages[-1].content
    
    # Sử dụng LLM để phân tích profile
    analysis_prompt = f"""
    Phân tích tin nhắn sau và trích xuất thông tin về người dùng:
    "{latest_message}"
    
    Trả về phân tích dưới dạng JSON với các trường:
    - name: tên người dùng (nếu có)
    - interests: sở thích/quan tâm
    - mood: tâm trạng
    - intent: ý định/mục đích
    """
    
    response = llm.invoke([HumanMessage(content=analysis_prompt)])
    
    # Cập nhật user profile (đơn giản hóa)
    current_profile = state.get("user_profile", {})
    current_profile["last_analysis"] = response.content
    current_profile["message_count"] = len(human_messages)
    
    return {
        "user_profile": current_profile,
        "processing_steps": state.get("processing_steps", []) + ["profile_analyzed"],
        "current_topic": "profile_analysis"
    }

def context_manager_node(state: ComplexState) -> Dict[str, Any]:
    """Quản lý context và lịch sử hội thoại"""
    print("📚 Managing conversation context...")
    
    # Cập nhật lịch sử
    current_history = state.get("conversation_history", [])
    
    # Thêm bản tóm tắt bước hiện tại
    summary = {
        "step": len(current_history) + 1,
        "topic": state.get("current_topic", "unknown"),
        "message_count": len(state.get("messages", [])),
        "user_profile_status": "analyzed" if state.get("user_profile") else "not_analyzed"
    }
    
    current_history.append(summary)
    
    return {
        "conversation_history": current_history,
        "processing_steps": state.get("processing_steps", []) + ["context_managed"],
        "metadata": {
            "total_steps": len(current_history),
            "last_update": "context_manager"
        }
    }

def response_generator_node(state: ComplexState) -> Dict[str, Any]:
    """Tạo phản hồi dựa trên toàn bộ state"""
    print("💬 Generating contextual response...")
    
    # Tạo context summary từ state
    profile_info = state.get("user_profile", {})
    history_info = state.get("conversation_history", [])
    processing_info = state.get("processing_steps", [])
    
    context_summary = f"""
    Thông tin context hiện tại:
    - User profile: {len(profile_info)} fields
    - Conversation history: {len(history_info)} steps
    - Processing steps: {', '.join(processing_info)}
    - Current topic: {state.get('current_topic', 'unknown')}
    """
    
    # Tạo phản hồi có ngữ cảnh
    prompt = f"""
    Bạn là một AI assistant thông minh. Dựa vào thông tin context sau:
    {context_summary}
    
    Hãy tạo một phản hồi thân thiện và có ích cho người dùng, 
    thể hiện rằng bạn đã hiểu context và có thể hỗ trợ tốt hơn.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    return {
        "messages": [response],
        "processing_steps": state.get("processing_steps", []) + ["response_generated"],
        "current_topic": "response_complete"
    }

# Tạo Complex State Graph
complex_workflow = StateGraph(ComplexState)

# Thêm nodes
complex_workflow.add_node("profile_analyzer", profile_analyzer_node)
complex_workflow.add_node("context_manager", context_manager_node)
complex_workflow.add_node("response_generator", response_generator_node)

# Entry point
complex_workflow.set_entry_point("profile_analyzer")

# Sequential edges
complex_workflow.add_edge("profile_analyzer", "context_manager")
complex_workflow.add_edge("context_manager", "response_generator")
complex_workflow.add_edge("response_generator", END)

# Compile
complex_app = complex_workflow.compile()

print("✅ Complex State Graph đã được tạo thành công!")

In [ ]:
# Test Complex State Graph
print("🚀 Test Complex State Management...\n")

test_message = "Xin chào! Tôi là Minh, đang học về AI và machine learning. Tôi muốn tìm hiểu về LangGraph."

initial_complex_state = {
    "messages": [HumanMessage(content=test_message)],
    "user_profile": {},
    "conversation_history": [],
    "current_topic": "introduction",
    "processing_steps": [],
    "metadata": {}
}

print(f"📝 Input: {test_message}\n")

try:
    result = complex_app.invoke(initial_complex_state)
    
    print("\n📊 State Analysis:")
    print(f"- Processing steps: {result.get('processing_steps', [])}")
    print(f"- Current topic: {result.get('current_topic', 'unknown')}")
    print(f"- Conversation history length: {len(result.get('conversation_history', []))}")
    print(f"- User profile fields: {len(result.get('user_profile', {}))}")
    print(f"- Total messages: {len(result.get('messages', []))}")
    
    print("\n👤 User Profile:")
    profile = result.get('user_profile', {})
    for key, value in profile.items():
        if key == 'last_analysis':
            print(f"- {key}: {value[:100]}..." if len(str(value)) > 100 else f"- {key}: {value}")
        else:
            print(f"- {key}: {value}")
    
    print("\n📚 Conversation History:")
    for step in result.get('conversation_history', []):
        print(f"- Step {step.get('step', '?')}: {step}")
    
    print("\n🤖 Final Response:")
    ai_messages = [msg for msg in result['messages'] if isinstance(msg, AIMessage)]
    if ai_messages:
        print(ai_messages[-1].content)
        
except Exception as e:
    print(f"❌ Lỗi: {str(e)}")

## Streaming và Async Execution

In [ ]:
# Demo streaming execution
print("🌊 Demo Streaming Execution...\n")

stream_state = {
    "messages": [HumanMessage(content="Giải thích về LangGraph một cách chi tiết")],
    "step_count": 0,
    "current_action": "starting"
}

print("📡 Streaming results:")
print("-" * 40)

try:
    # Stream từng bước
    for step_result in simple_app.stream(stream_state):
        print(f"🔸 Stream step: {list(step_result.keys())}")
        
        for node_name, node_result in step_result.items():
            print(f"   Node '{node_name}':")
            print(f"   - Current action: {node_result.get('current_action', 'unknown')}")
            print(f"   - Step count: {node_result.get('step_count', 0)}")
            
            # Hiển thị tin nhắn nếu có
            if 'messages' in node_result and node_result['messages']:
                last_msg = node_result['messages'][-1]
                if hasattr(last_msg, 'content'):
                    content = last_msg.content[:100] + "..." if len(last_msg.content) > 100 else last_msg.content
                    print(f"   - Message: {content}")
            print()
            
except Exception as e:
    print(f"❌ Streaming error: {str(e)}")

print("✅ Streaming demo completed!")

## Giải thích & Phân tích

### 🎯 Các thành phần đã sử dụng

#### 1. **StateGraph**
- **Mục đích**: Container chính để quản lý workflow với state
- **Sử dụng**: `StateGraph(StateSchema)` với TypedDict schema
- **Ưu điểm**: Type safety, state persistence giữa các nodes

#### 2. **add_node()**
- **Syntax**: `workflow.add_node("name", function)`
- **Function signature**: `(state: StateType) -> Dict[str, Any]`
- **Return**: Partial state update (merged với existing state)

#### 3. **add_edge() & add_conditional_edges()**
- **Simple edge**: `add_edge("from", "to")` - luôn chuyển
- **Conditional**: `add_conditional_edges("from", condition_func, mapping)`
- **Condition function**: Nhận state, trả về string key

#### 4. **compile() & invoke()**
- **compile()**: Tạo runnable graph từ workflow definition
- **invoke()**: Thực thi đồng bộ, return final state
- **stream()**: Thực thi với streaming, yield intermediate results

### 🔧 Pattern phổ biến

1. **State Management**
   ```python
   class MyState(TypedDict):
       messages: Annotated[list, add_messages]  # Auto-merge
       custom_field: Any  # Overwrite
   ```

2. **Tool Integration**
   ```python
   llm_with_tools = llm.bind_tools([tool1, tool2])
   # Trong node: check tool_calls, execute, return results
   ```

3. **Conditional Logic**
   ```python
   def should_continue(state):
       return "continue" if condition else "end"
   ```

### ⚡ Performance Notes
- **State size**: Giữ state nhỏ gọn, tránh lưu data lớn
- **Node functions**: Stateless, pure functions khi có thể
- **Error handling**: Wrap trong try-catch, return error state
- **Streaming**: Sử dụng cho long-running processes

## Tài liệu tham khảo

### 📚 Official Documentation
- **LangGraph API Reference**: https://langchain-ai.github.io/langgraph/reference/
- **LangGraph Tutorials**: https://langchain-ai.github.io/langgraph/tutorials/
- **StateGraph API**: https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.StateGraph
- **Message Handling**: https://langchain-ai.github.io/langgraph/concepts/low_level/#messages

### 🔗 Related Resources
- **LangChain Tools**: https://python.langchain.com/docs/modules/tools/
- **Anthropic Claude**: https://docs.anthropic.com/claude/docs
- **TypedDict Documentation**: https://docs.python.org/3/library/typing.html#typing.TypedDict

### 🛠️ Code Examples
- **LangGraph Examples**: https://github.com/langchain-ai/langgraph/tree/main/examples
- **Multi-Agent Systems**: https://langchain-ai.github.io/langgraph/tutorials/multi_agent/
- **Tool Calling**: https://langchain-ai.github.io/langgraph/tutorials/tool-calling/

### 📖 Best Practices
- **State Design Patterns**: Keep state minimal and focused
- **Error Handling**: Always handle exceptions in nodes
- **Performance**: Use streaming for long-running processes
- **Testing**: Test individual nodes before integration

## Kết luận & Bước tiếp theo

### 🎓 Những gì đã học
1. **Cấu trúc LangGraph**: StateGraph, Nodes, Edges, State management
2. **API Reference**: Cách sử dụng tài liệu chính thức và tìm kiếm thông tin
3. **Thực hành**: 3 ví dụ từ đơn giản đến phức tạp
4. **Tool Integration**: Kết hợp LLM với external tools
5. **State Management**: Quản lý state phức tạp với TypedDict
6. **Streaming**: Thực thi bất đồng bộ và real-time updates

### 🚀 Bước tiếp theo
1. **Khám phá API Reference sâu hơn**:
   - Đọc documentation của các class chưa sử dụng
   - Thử nghiệm với các parameters khác nhau
   - Tìm hiểu về error handling patterns

2. **Thực hành nâng cao**:
   - Xây dựng multi-agent systems
   - Tích hợp với databases và external APIs
   - Implement custom state reducers

3. **Production readiness**:
   - Error handling và logging
   - Performance optimization
   - Testing strategies

4. **Tích hợp ecosystem**:
   - LangSmith cho monitoring
   - LangServe cho deployment
   - Custom tools và integrations

### 💡 Tips cho việc sử dụng API Reference
- **Bookmark** các trang quan trọng
- **Đọc examples** trước khi implement
- **Check version compatibility** khi update
- **Contribute back** với feedback và bug reports

---

**Happy coding với LangGraph! 🎉**